### Import python library

In [1]:
import numpy as np
import pandas as pd

### 1. Read data: bigcon_final_dataset.csv필요

In [110]:
data = pd.read_csv("bigcon_cumulate_final_dataset.csv")

In [111]:
team = list(set(data.T_ID))

### 2. Train 전처리

In [112]:
train = data[data["IDX"]!=777]

In [113]:
sel_col = ["T_ID","IDX","MERGE_IDX","PCT","ERA","AVG",'YEAR']
train_shift = train.loc[:,sel_col]
shift_col = sorted(list(set(list(train.columns))-set(['T_ID','YEAR'])))

for i in range(1,2):
    for c in shift_col:
        train_shift.loc[:,'shift_{}'.format(c)] = train.loc[:,c].shift(i)

In [114]:
train_shift.dropna(inplace = True)
ts_idx = list(train_shift.index)

drop_idx = []
for i in ts_idx:
    idx_num = train_shift.loc[i,'IDX']
    if idx_num <= train_shift.loc[i,'shift_IDX']:
        drop_idx.append(i)

train_s_df = train_shift.drop(drop_idx).reset_index(drop = True)  

train_s_drop_df = train_s_df.drop(["IDX","MERGE_IDX","shift_IDX","shift_MERGE_IDX","ERA","AVG","PCT"],axis = 1)

### 3. Test 전처리

In [115]:
test = data[data["IDX"]==777]

In [116]:
tmp1619 = data[data["IDX"]==5]
test = pd.concat([test,tmp1619],axis=0)

tmp2020 = data[data["YEAR"]==2020]
for t in team:
    tmp = tmp2020[tmp2020["T_ID"]==t]
    mer_idx = max(tmp.MERGE_IDX)
    test = pd.concat([test,tmp[tmp["MERGE_IDX"]==mer_idx]], axis = 0)
    

test = test.sort_values(by=["YEAR","T_ID","IDX"]).reset_index(drop=True)
    

In [117]:
test_shift = test.loc[:,sel_col]
shift_col = sorted(list(set(list(test.columns))-set(['T_ID','YEAR'])))

for i in range(1,2):
    for c in shift_col:
        test_shift.loc[:,'shift_{}'.format(c)] = test.loc[:,c].shift(i)

In [118]:
test_s_df = test_shift[test_shift["IDX"]==777].reset_index(drop=True)
test_s_drop_df = test_s_df.drop(["IDX","MERGE_IDX","shift_IDX","shift_MERGE_IDX","ERA","AVG","PCT"],axis = 1)

### 4.  split data  -> PCT/AVG/ERA dataset 가공

In [119]:
common_list = ['T_ID', 'YEAR', 'HEADER_NO','PCT']
hitter = ['AVG', 'OBP', 'SLG', 'OPS', 'RC', 'GPA', 'BA', 'SECA', 
          'TA', 'XR', 'ISO', 'wOBA', 'P_HRA_RT', 'DER']
pitcher = ['WHIP', 'LOB', 'ERA', 'FIP', 'H_9', 'K_9', 'BB_9', 
           'HR_9', 'oAVG', 'oSLG', 'oOBP', 'P_WHIP_RT', 'P2_WHIP_RT', 'CB_WHIP_RT','oISO', 'oOPS']

vs_hitter = ['vs_AVG', 'vs_OBP', 'vs_SLG', 'vs_OPS', 'vs_RC', 'vs_GPA', 'vs_BA', 'vs_SECA', 
             'vs_TA', 'vs_XR', 'vs_ISO', 'vs_wOBA', 'vs_P_HRA_RT', 'vs_DER']
vs_pitcher = ['vs_WHIP', 'vs_LOB', 'vs_ERA', 'vs_FIP', 'vs_H_9', 'vs_K_9', 'vs_BB_9', 
              'vs_HR_9', 'vs_oAVG', 'vs_oSLG', 'vs_oOBP', 'vs_P_WHIP_RT', 'vs_P2_WHIP_RT', 'vs_CB_WHIP_RT'
             ,'vs_oISO', 'vs_oOPS']

avg_col = common_list + hitter + vs_pitcher
era_col = common_list + pitcher + vs_hitter

In [120]:
train_col = list(train_s_drop_df.columns)
sel_avg_lst =train_col[:2]
sel_era_lst = train_col[:2]

for i in train_col[2:]:
    for k in avg_col:
        if k in i:
            sel_avg_lst.append(i)
            break
    
for i in train_col[2:]:
    for k in era_col:
        if k in i:
            sel_era_lst.append(i)
            break


In [121]:
# 기존에서 제외된 column 목록
list(set(list(data.columns))- set(hitter+pitcher + vs_hitter+vs_pitcher + common_list))

['IDX', 'MERGE_IDX']

### 5-1. 최종 데이터셋 생성

In [122]:
#승률
PCT_train_X = train_s_drop_df
PCT_test_X = test_s_drop_df

In [123]:
# 타율
AVG_train_X = train_s_drop_df[sel_avg_lst]
AVG_test_X = test_s_drop_df[sel_avg_lst]

In [124]:
# 방어율
ERA_train_X = train_s_drop_df[sel_era_lst]
ERA_test_X = test_s_drop_df[sel_era_lst]

### 5-2. column 명 변경

In [125]:
col = list(PCT_train_X.columns)[2:]
col_dict = {}

for i in col:
    col_dict[i] = i.replace('shift','c')
    
PCT_train_X.rename(columns = col_dict, inplace = True)
PCT_test_X.rename(columns = col_dict, inplace = True)

In [126]:
col = list(AVG_train_X.columns)[2:]
col_dict = {}

for i in col:
    col_dict[i] = i.replace('shift','c')
    
AVG_train_X.rename(columns = col_dict, inplace = True)
AVG_test_X.rename(columns = col_dict, inplace = True)

In [127]:
col = list(ERA_train_X.columns)[2:]
col_dict = {}

for i in col:
    col_dict[i] = i.replace('shift','c')
    

ERA_train_X.rename(columns = col_dict, inplace = True)
ERA_test_X.rename(columns = col_dict, inplace = True)

### 6. 저장

In [128]:
PCT_train_X.to_csv("PCT/PCT_c_train_X.csv",index=False)
PCT_test_X.to_csv("PCT/PCT_c_test_X.csv",index=False)

In [129]:
AVG_train_X.to_csv("AVG/AVG_c_train_X.csv",index=False)
AVG_test_X.to_csv("AVG/AVG_c_test_X.csv",index=False)

In [130]:
ERA_train_X.to_csv("ERA/ERA_c_train_X.csv",index=False)
ERA_test_X.to_csv("ERA/ERA_c_test_X.csv",index=False)